In [ ]:
! pip install panns-inference
!pip install keytotext --upgrade

In [ ]:
from keytotext import pipeline
nlp = pipeline("mrm8488/t5-base-finetuned-common_gen")
k2t_new = pipeline("k2t-new")

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import librosa
import panns_inference
from panns_inference import AudioTagging, SoundEventDetection, labels

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/clotho_captions_evaluation.csv')
folder_path = '/content/drive/MyDrive/clotho test'

candidates1 = []
candidates2 = []
references = []
for sample in os.listdir(folder_path):
  sample = os.listdir(folder_path)[j]
  audio_path = os.path.join(folder_path, sample)
  (audio, _) = librosa.core.load(audio_path, sr=32000, mono=True)
  audio = audio[None, :]
  at = AudioTagging(checkpoint_path=None, device='cpu')
  (clipwise_output, embedding) = at.inference(audio)
  sorted_indexes = np.argsort(clipwise_output[0])[::-1]
  keywords = []
  for i in range(3):
    keywords.append(np.array(labels)[sorted_indexes[i]])
  sentence1 = k2t_new(keywords)
  candidates1.append(str(sentence1).split())
  sentence2 = nlp(keywords)
  candidates2.append(str(sentence2).split())
  reference = []
  test_sample = df[df.file_name == sample]
  reference.append(str(test_sample.caption_1.to_list()[0]).split())
  reference.append(str(test_sample.caption_2.to_list()[0]).split())
  reference.append(str(test_sample.caption_3.to_list()[0]).split())
  reference.append(str(test_sample.caption_4.to_list()[0]).split())
  reference.append(str(test_sample.caption_5.to_list()[0]).split())
  references.append(reference)

Clotho Caption:

In [ ]:
print(' '.join(references[8][1]))

A fork is being banged onto a drinking glass.


Prediction:

In [ ]:
print(' '.join(candidates1[8]))

A man is clinking a glass in a room.


In [ ]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

In [ ]:
print('bleu1: ', corpus_bleu(references, candidates1, weights=(1, 0, 0, 0)))
print('bleu2: ', corpus_bleu(references, candidates1, weights=(0, 1, 0, 0)))

bleu1:  0.35570755629562534
bleu2:  0.04294820732298278


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
